In [ ]:
## setup
## TODO: move all under bird2022wheels
!pip install ../input/birds-inference-pip-wheels/torchaudio-0.8.1-cp37-cp37m-manylinux1_x86_64.whl ../input/birds-inference-pip-wheels/torch-1.8.1-cp37-cp37m-manylinux1_x86_64.whl
!pip install ../input/birds-inference-pip-wheels/audiomentations-0.16.0-py3-none-any.whl --no-index --no-deps
!pip install ../input/birds-inference-pip-wheels/torchlibrosa-0.0.9-py3-none-any.whl --no-index --no-deps
!pip install ../input/birds2022wheels/nnAudio-0.3.1-py3-none-any.whl
!cp -r ../input/timmlatest ../working/timmlatest
!pip install -U ../working/timmlatest
!rm -rf ../working/timmlatest

In [4]:
## https://github.com/Selimonder/birdclef2022/
import os, sys, glob, math

os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

github_folder = "."
sys.path.append(github_folder)

In [5]:
import cv2
import gc
import torch
import librosa
import argparse, warnings

import numpy as np
import pandas as pd
import IPython.display as ipd

import zoo
from training.config import load_config
from training.datasets import BirdDatasetOOF


cv2.ocl.setUseOpenCL(False)
cv2.setNumThreads(0)

from tqdm import tqdm
from torch.utils.data import DataLoader

warnings.simplefilter("ignore")

## models

In [15]:
def load_model(conf_path: str, weights_path: str, prefix: str, suffix: str, fold: int, to_device: bool = True):
    conf = load_config(conf_path)
    conf['encoder_params']['pretrained'] = False
    
    snapshot_name = "{}{}_{}_{}_{}".format(prefix, conf["network"], conf["encoder_params"]["encoder"], fold, suffix)
    weights_path = os.path.join(weights_path, snapshot_name)
    print(weights_path)
    model = zoo.__dict__[conf["network"]](**conf["encoder_params"])
    model = torch.nn.DataParallel(model).cuda()
    print("=> loading checkpoint '{}''".format(weights_path))
    checkpoint = torch.load(weights_path, map_location="cpu")
    print("epoch", checkpoint["epoch"])
    model.load_state_dict(checkpoint["state_dict"])
    model.eval()
    if to_device: model.cuda()
    return model

models = []

## best only for now
suffixes = ["lb"]#, "f1_score" "last"]
folds    = 5

for i in range(folds):
    for sx in suffixes:
        try:
            model = load_model(conf_path    = f"{github_folder}/configs/cls_nf0_v5.json",
                               weights_path = f"{github_folder}/weights/nfnet-baseline-bs16",
                               prefix       = "nf0_v5_",
                               suffix       = sx,
                               fold         = i)
            models.append(model)
        except Exception as e:
            print(f"model not found", e)

./weights/nfnet-baseline-bs16/nf0_v5_TimmClassifier_eca_nfnet_l0_0_lb
initing CLS features model 15 duration...
pretrained model...
{'in_chans': 1, 'drop_path_rate': 0.2, 'drop_rate': 0.5}
=> loading checkpoint './weights/nfnet-baseline-bs16/nf0_v5_TimmClassifier_eca_nfnet_l0_0_lb''
epoch 18
./weights/nfnet-baseline-bs16/nf0_v5_TimmClassifier_eca_nfnet_l0_1_lb
initing CLS features model 15 duration...
pretrained model...
{'in_chans': 1, 'drop_path_rate': 0.2, 'drop_rate': 0.5}
=> loading checkpoint './weights/nfnet-baseline-bs16/nf0_v5_TimmClassifier_eca_nfnet_l0_1_lb''
epoch 14
./weights/nfnet-baseline-bs16/nf0_v5_TimmClassifier_eca_nfnet_l0_2_lb
initing CLS features model 15 duration...
pretrained model...
{'in_chans': 1, 'drop_path_rate': 0.2, 'drop_rate': 0.5}
=> loading checkpoint './weights/nfnet-baseline-bs16/nf0_v5_TimmClassifier_eca_nfnet_l0_2_lb''
epoch 15
./weights/nfnet-baseline-bs16/nf0_v5_TimmClassifier_eca_nfnet_l0_3_lb
initing CLS features model 15 duration...
pretraine

In [17]:
len(models)

5

## infer

In [21]:
CLASSES = "abethr1 abhori1 abythr1 afbfly1 afdfly1 afecuc1 affeag1 afgfly1 afghor1 afmdov1 afpfly1 afpkin1 afpwag1 afrgos1 afrgrp1 afrjac1 afrthr1 amesun2 augbuz1 bagwea1 barswa bawhor2 bawman1 bcbeat1 beasun2 bkctch1 bkfruw1 blacra1 blacuc1 blakit1 blaplo1 blbpuf2 blcapa2 blfbus1 blhgon1 blhher1 blksaw1 blnmou1 blnwea1 bltapa1 bltbar1 bltori1 blwlap1 brcale1 brcsta1 brctch1 brcwea1 brican1 brobab1 broman1 brosun1 brrwhe3 brtcha1 brubru1 brwwar1 bswdov1 btweye2 bubwar2 butapa1 cabgre1 carcha1 carwoo1 categr ccbeat1 chespa1 chewea1 chibat1 chtapa3 chucis1 cibwar1 cohmar1 colsun2 combul2 combuz1 comsan crefra2 crheag1 crohor1 darbar1 darter3 didcuc1 dotbar1 dutdov1 easmog1 eaywag1 edcsun3 egygoo equaka1 eswdov1 eubeat1 fatrav1 fatwid1 fislov1 fotdro5 gabgos2 gargan gbesta1 gnbcam2 gnhsun1 gobbun1 gobsta5 gobwea1 golher1 grbcam1 grccra1 grecor greegr grewoo2 grwpyt1 gryapa1 grywrw1 gybfis1 gycwar3 gyhbus1 gyhkin1 gyhneg1 gyhspa1 gytbar1 hadibi1 hamerk1 hartur1 helgui hipbab1 hoopoe huncis1 hunsun2 joygre1 kerspa2 klacuc1 kvbsun1 laudov1 lawgol lesmaw1 lessts1 libeat1 litegr litswi1 litwea1 loceag1 lotcor1 lotlap1 luebus1 mabeat1 macshr1 malkin1 marsto1 marsun2 mcptit1 meypar1 moccha1 mouwag1 ndcsun2 nobfly1 norbro1 norcro1 norfis1 norpuf1 nubwoo1 pabspa1 palfly2 palpri1 piecro1 piekin1 pitwhy purgre2 pygbat1 quailf1 ratcis1 raybar1 rbsrob1 rebfir2 rebhor1 reboxp1 reccor reccuc1 reedov1 refbar2 refcro1 reftin1 refwar2 rehblu1 rehwea1 reisee2 rerswa1 rewsta1 rindov rocmar2 rostur1 ruegls1 rufcha2 sacibi2 sccsun2 scrcha1 scthon1 shesta1 sichor1 sincis1 slbgre1 slcbou1 sltnig1 sobfly1 somgre1 somtit4 soucit1 soufis1 spemou2 spepig1 spewea1 spfbar1 spfwea1 spmthr1 spwlap1 squher1 strher strsee1 stusta1 subbus1 supsta1 tacsun1 tafpri1 tamdov1 thrnig1 trobou1 varsun2 vibsta2 vilwea1 vimwea1 walsta1 wbgbir1 wbrcha2 wbswea1 wfbeat1 whbcan1 whbcou1 whbcro2 whbtit5 whbwea1 whbwhe3 whcpri2 whctur2 wheslf1 whhsaw1 whihel1 whrshr1 witswa1 wlwwar wookin1 woosan wtbeat1 yebapa1 yebbar1 yebduc1 yebere1 yebgre1 yebsto1 yeccan1 yefcan yelbis1 yenspu1 yertin1 yesbar1 yespet1 yetgre1 yewgre1".split()
N_CLASS = len(CLASSES)

In [22]:

def prepare_clip(fpath, frame_length, sample_rate):
    """
    Prepare audio clip for inference
    """
    infer_frame_length = frame_length
    batch = {"wav_tensors": [], "end_times": []}
    
    waveform, sample_rate = librosa.load(fpath, sr=sample_rate, mono=True)
    n_parts = math.ceil(len(waveform) / int(infer_frame_length * sample_rate))
    
    for seg_idx in range(n_parts): 
        end_time = (seg_idx + 1) * frame_length
        seg_wav  = waveform[(end_time*sample_rate)-(sample_rate*frame_length):end_time*sample_rate]
        
        wav_tensor = torch.from_numpy(seg_wav)
        
        if len(wav_tensor) == frame_length * sample_rate:
            batch["wav_tensors"].append(wav_tensor.unsqueeze(0))
        elif len(wav_tensor) < frame_length * sample_rate:
            wav_tensor = torch.nn.functional.pad(wav_tensor, (0, (frame_length * sample_rate) - len(wav_tensor)))
            batch["wav_tensors"].append(wav_tensor.unsqueeze(0))
        elif len(wav_tensor) > frame_length * sample_rate:
            wav_tensor = wav_tensor[:(frame_length * sample_rate)]
            batch["wav_tensors"].append(wav_tensor.unsqueeze(0))
            
        batch["end_times"].append(end_time)
    batch["wav_tensors"] = torch.stack(batch["wav_tensors"]).cuda()
    return batch, n_parts, len(waveform)

@torch.no_grad()
def predict_clip(models, batch, n_parts, frame_length):
    preds = np.zeros([len(models), n_parts, N_CLASS])
    for m_idx, model in enumerate(models):
        with torch.cuda.amp.autocast():
            preds[m_idx] = model(batch["wav_tensors"], is_test=True)["logit"].sigmoid().cpu().numpy()
    return preds.max(0) ## max by model    
    

In [23]:
len(models)

5

In [30]:
%%time


@torch.no_grad()
def generate_preds():
    debug = False
    preds        = []
    test_files   = sorted(glob.glob("/kaggle/input/birdclef-2023/test_soundscapes/*.ogg")) 
    sample_rate  = 32000
    
    scored_birds = np.array(CLASSES)

    frame_length = 5
    infer_frame_length = 5

    print("test file num:", len(test_files))
    for fpath in test_files:
        file_id = os.path.basename(fpath).replace(".ogg", "")
        batch, n_parts, len_waveform = prepare_clip(fpath, frame_length, sample_rate)
        clip_preds = predict_clip(models, batch, n_parts, frame_length)
        preds.append(clip_preds)
        
        ## switch to infer frame length
        n_parts_sub = math.ceil(len_waveform / int(infer_frame_length * sample_rate))
        clip_preds = np.array_split(clip_preds, n_parts_sub, axis=0)
        
                
    prob_array = np.array(preds)
    
    scored_bird_name2idx = {}
    for i, x in enumerate(scored_birds):
        scored_bird_name2idx[x] = i

    return prob_array, scored_bird_name2idx

prob_array, scored_bird_name2idx = generate_preds()
# del models
# gc.collect()
# torch.cuda.empty_cache()

test file num: 1
CPU times: user 710 ms, sys: 9.83 ms, total: 720 ms
Wall time: 719 ms


In [27]:
prob_array.shape

(1, 120, 264)

----

In [28]:
%cd /kaggle/input/birdnetgit

[Errno 2] No such file or directory: '/kaggle/input/birdnetgit'
/home/mobius/code/kaggle/BLEF/rank1/birdclef-2022


In [29]:
import os
import sys
import json
import math
import glob
import librosa
import operator
import argparse
import datetime
import traceback

from multiprocessing import Pool, freeze_support

import numpy as np

import config as cfg
import audio
import model
from analyze import *

import pandas as pd
import numpy as np

ModuleNotFoundError: No module named 'config'

In [ ]:
df = pd.read_csv("../birdclef-2022/eBird_Taxonomy_v2021.csv")
s2id = lambda x: df[(df['PRIMARY_COM_NAME'] == x)].SPECIES_CODE.tolist()[0]

cfg.MODEL_PATH = '/kaggle/input/birdnetgit/checkpoints/V2.1/BirdNET_GLOBAL_2K_V2.1_Model_FP32.tflite'
cfg.MDATA_MODEL_PATH = '/kaggle/input/birdnetgit/checkpoints/V2.1/BirdNET_GLOBAL_2K_V2.1_MData_Model_FP32.tflite'
cfg.LABELS_FILE = '/kaggle/input/birdnetgit/checkpoints/V2.1/BirdNET_GLOBAL_2K_V2.1_Labels.txt'
cfg.TRANSLATED_LABELS_PATH = '/kaggle/input/birdnetgit/labels/V2.1'

cfg.SIG_LENGTH = 5
cfg.SIG_OVERLAP = 0 
cfg.SIG_MINLEN = 5 

# Load eBird codes, labels

def loadCodes(CODEC_FILE):

    with open(CODEC_FILE, 'r') as cfile:
        codes = json.load(cfile)

    return codes

cfg.CODES = loadCodes("./eBird_taxonomy_codes_2021E.json")
def loadLabels(labels_file):

    labels = []
    with open(labels_file, 'r') as lfile:
        for line in lfile.readlines():
            labels.append(line.replace('\n', ''))    

    return labels

def prepare_clip(fpath):
    """
    Prepare audio clip for inference
    """
    sample_rate = cfg.SAMPLE_RATE
    frame_length = cfg.SIG_LENGTH
    infer_frame_length = frame_length
    
    sample_rate = 48000
    chunks = []
    
    waveform, sample_rate = librosa.load(fpath, sr=sample_rate, mono=True, res_type='kaiser_fast')
    n_parts = math.ceil(len(waveform) / int(infer_frame_length * sample_rate))
    
    for seg_idx in range(n_parts): 
        end_time = (seg_idx + 1) * frame_length
        chunk  = waveform[(end_time*sample_rate)-(sample_rate*frame_length):end_time*sample_rate]
                
        if len(chunk) == frame_length * sample_rate:
            chunks.append(chunk)
        elif len(chunk) < frame_length * sample_rate:
            chunk = np.pad(chunk, (0, (frame_length * sample_rate) - len(chunk)))
            chunks.append(chunk)
        elif len(chunk) > frame_length * sample_rate:
            chunk = chunk[:(frame_length * sample_rate)]
            chunks.append(chunk)
            
    return chunks


scored_birds = np.array(['akiapo', 'aniani', 'apapan', 'barpet', 'crehon', 'elepai', 'ercfra',
                          'hawama', 'hawcre', 'hawgoo', 'hawhaw', 'hawpet1', 'houfin', 'iiwi',
                          'jabwar', 'maupar', 'omao', 'puaioh', 'skylar', 'warwhe1', 'yefcan'])

cfg.LABELS = loadLabels(cfg.LABELS_FILE)

model.loadModel()



In [ ]:
bird_dict = {}
for i, lb in enumerate(cfg.LABELS):
    try:
        lbb = s2id(lb.split("_")[-1])
        if lbb in scored_birds:
            bird_dict[lbb] = i
    except Exception as e:
        continue
    
bird_dict["aniani"] = 0

sorted_bd = dict(sorted(bird_dict.items(), key=lambda item: item[0]))
get_idx = list(sorted_bd.values())

In [ ]:
# def predict_single(test_path)

def generate_preds_bn():
    test_files   = sorted(glob.glob("/kaggle/input/birdclef-2022/test_soundscapes/*.ogg"))
    
    preds = []
    file_ids = []
    
    for fpath in test_files:
        chunks = prepare_clip(fpath)
        pred = predict(chunks)[:, get_idx]
        preds.append(pred)
        file_id = os.path.basename(fpath).replace(".ogg", "")
        file_ids.append(file_id)
        
    preds = np.array(preds)
    return preds, file_ids
        

In [ ]:
%%time
preds_bn, file_ids = generate_preds_bn()


In [ ]:
preds_bn[:, :, 1] = prob_array[:, :, 1]

In [ ]:
## fix aniani
avg_preds = (2 * preds_bn + prob_array) / 3

avg_preds.shape

In [ ]:
submission   = []
infer_frame_length = 5
threshold = 0.2

for clip_preds, file_id in zip(avg_preds, file_ids):
    for frame_idx, pred in enumerate(clip_preds):
        end_time = (frame_idx + 1) * infer_frame_length
        for bi, bird in enumerate(scored_birds):
            submission.append({
                "row_id": f"{file_id}_{bird}_{end_time}",
                "target": pred[bi] > threshold,
            })

In [ ]:
%cd /kaggle/working

In [ ]:
df_submission = pd.DataFrame(submission).set_index("row_id")
df_submission.to_csv("submission.csv")

In [ ]:
print(f"""
total rows : {len(df_submission)}
activated  : {len(df_submission[(df_submission.target==True)])}
""")

df_submission[(df_submission.target==True)].head()